## Data animation

### Part 1 - Static Data Visualisation

1) Importing libraries and dataset from ```auto-mpg.csv```.

In [10]:
%matplotlib qt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [11]:
df = pd.read_csv("auto-mpg.csv")
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,1,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,2,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,3,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,4,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,5,1,ford torino


In [13]:
fig, ax = plt.subplots()

plt.scatter(df.iloc[:,0], df.iloc[:,3], color="darkorange", edgecolor='black', zorder=1)

ax.set_xlabel('Miles per Gallon (mpg)')
ax.set_ylabel('Horsepower (hp)')
ax.set_title(' Power versus fuel efficiency')

plt.show()

2) Constructing the set of rules for data to be used in plot of power against fuel efficiency.

In [159]:
#initialised the array containing power and fuel efficiency
power = df.iloc[:, 3]
fuel = df.iloc[:, 0]
#initialised empty array to store index of non-dominated cars 
dominate = []
#initialised empty array to store index of dominated cars
notdom = []
#empty array to store the entire index 
coll = []

for i in range(len(df)):
    #extract the power and fuel efficiency of the respective car
    p = df.iloc[i, 3]
    f = df.iloc[i, 0]
    #initialised the count for cars dominated by it as zero
    k = 0
    #store the car's index into coll
    coll.append(i)
    for j in coll:
        #if the car's power or car's fuel efficiency is higher than any other cars in coll do
        if ((p > power[j]) | (f > fuel[j])):
            #increment the count for cars dominated by it
            k += 1
            #if the car's power or car's fuel efficiency is higher than any other cars in dominate do
            if ((p >= power[j]) & (f >= fuel[j])) & (j in dominate):
                #remove the other cars from dominate
                dominate.remove(j)
                #add the other cars into notdom
                notdom.append(j)

    #if the count for k is less than the cars in coll do
    if (k < len(coll) - 1):
        #label the cars as dominated 
        notdom.append(i)
    else:
        #label the cars as non-dominated
        dominate.append(i)

3) Make sure that the number of dominated and non-dominated cars equal the total number of cars.

In [160]:
num_dominate = len(dominate)
num_notdom = len(notdom)
total = len(df)
print(r"total : ", total, ", non dominated : ", num_dominate, ", dominated : ", num_notdom)

total :  392 , non dominated :  14 , dominated :  378


4) Plotting the results for the record of power against fuel efficiency.

In [164]:
#initialised the plot
fig, ax = plt.subplots()

#plot non-dominated cars
for i in dominate:
    #plot the dominating region
    poly = plt.Rectangle((0, 0), df.iloc[i, 0], df.iloc[i, 3], fc='lightgrey', alpha=1.0, zorder=0)
    plt.gca().add_patch(poly)

#plot dominated cars
plt.scatter(df.iloc[notdom,0], df.iloc[notdom,3], color="darkorange", edgecolor='black', zorder=1, label="dominated")
#plot non-dominated cars
plt.scatter(df.iloc[dominate,0], df.iloc[dominate,3], color="cyan", edgecolor='black', zorder=2, label="non-dominated")

#plot the results
ax.set_xlabel('Miles per Gallon (mpg)')
ax.set_ylabel('Horsepower (hp)')
ax.set_title(' Power versus fuel efficiency')
plt.legend()
plt.show()

### Part 2 : Animation

1) Importing libraries from ```animation``` in ```matplotlib```.

In [149]:
from matplotlib.animation import FuncAnimation
from matplotlib.animation import ArtistAnimation

2) Set a few constraints and global imput for animations.

In [150]:
#total number of cars
max_cars = len(df)

In [151]:
#index of car
num_cars = 0
#initialised the array containing power and fuel efficiency
power = df.iloc[:, 3]
fuel = df.iloc[:, 0]
#initialised empty array to store index of non-dominated cars 
dominate = []
#initialised empty array to store index of dominated cars
notdom = []
#empty array to store the entire index 
coll = []

3) Defining function ```cars``` that sets the rules for dominated and non-dominated cars based on Step (2) in Part 1.

In [152]:
def cars(num_cars):
    p = df.iloc[num_cars, 3]
    f = df.iloc[num_cars, 0]
    k = 0
    coll.append(num_cars)
    for j in coll:
        if (p > power[j]) | (f > fuel[j]):
            k += 1
            if ((p >= power[j]) & (f >= fuel[j])) & (j in dominate):
                dominate.remove(j)
                notdom.append(j)

    if (k < len(coll) - 1):
        notdom.append(num_cars)
    else:
        dominate.append(num_cars)

4) Animate the results from Part 1 using ```Funcanimation``` and function ```cars```.

In [153]:
#initialised the figure to plot the results
fig, ax = plt.subplots(figsize=(11, 9))
#some labels for the plot
plt.xlim((0, df["mpg"].max()))
plt.ylim((0, df["horsepower"].max()))
ax.set_xlabel('Miles per Gallon (mpg)')
ax.set_ylabel('Horsepower (hp)')
ax.set_title(' Power versus fuel efficiency')

#function to animate output of function cars
def animate(frameNum):
    #access the global variables
    global num_cars, dominate, notdom, df, power, fuel, coll
    #if the index for the cars is less than the total number of cars do
    if (num_cars < max_cars):
        #run function cars with the respective cars
        cars(num_cars) 
        
        #clear the plot
        ax.clear()
        #label the plot
        ax.set_xlabel('Miles per Gallon (mpg)')
        ax.set_ylabel('Horsepower (hp)')
        ax.set_title(' Power versus fuel efficiency')
    
        #plot the dominating region
        for i in dominate:
            poly = plt.Rectangle((0, 0), df.iloc[i, 0], df.iloc[i, 3], fc='lightgrey', alpha=1.0, zorder=0)
            plt.gca().add_patch(poly)
            
        #plot the dominated cars
        plt.scatter(df.iloc[notdom,0], df.iloc[notdom,3], color="darkorange", edgecolor='black', zorder=1, label="dominated")
        #plot the non-dominated cars
        plt.scatter(df.iloc[dominate,0], df.iloc[dominate,3], color="cyan", edgecolor='black', zorder=2, label="non-dominated")
        
        plt.legend()
        #increment the index of cars
        num_cars += 1
        
# Create the animation. Note that we must keep a handle to it, i.e. say "anim = ", otherwise it will be garbage collected.
anim = FuncAnimation(fig, animate, repeat=False, blit=False, frames=max_cars, interval=50)

#uncomment the below line to save the animation
#anim.save('cars_dominate.mp4', fps=10)
plt.show()